# OpenAI Agent 

OpenAI 플랫폼을 사용하여 사용자를 대신해 행동(예: 컴퓨터 제어 등)을 수행할 수 있는 **에이전트(Agent)** 를 구축할 수 있습니다.  
Python용 **Agent SDK**를 사용하면 백엔드에서 이러한 에이전트의 **오케스트레이션(조율) 로직**을 만들 수 있습니다.

**OpenAI Agents SDK**는 **에이전트 기반 AI 애플리케이션**을 개발할 수 있게 해주는 도구입니다.  

**핵심 구성 요소**  

- **에이전트 (Agents)**:  
  LLM에 **지시문(instructions)** 과 **도구(tools)** 를 장착한 실행 단위  
- **핸드오프 (Handoffs)**:  
  특정 작업을 다른 에이전트에게 **위임**할 수 있도록 해줌  
- **가드레일 (Guardrails)**:  
  에이전트에게 전달되는 입력을 **검증**하는 장치

### 주요 기능 요약:

- **에이전트 루프 (Agent Loop)**:  
  도구 실행 → 결과 전달 → LLM 호출 → 반복 실행 → 완료까지 자동 처리

- **핸드오프 (Handoffs)**:  
  여러 에이전트 간의 **협업과 위임**을 유연하게 처리 가능

- **가드레일 (Guardrails)**:  
  에이전트 입력을 **사전 검사/검증**하여, 조건을 만족하지 않으면 **조기 종료 가능**

- **함수 기반 도구 (Function Tools)**:  
  Python 함수 하나를 **자동으로 에이전트 도구로 변환**,  
  **Pydantic 기반 스키마 자동 생성** 및 검증 포함

- **추적(Tracing)**:  
  워크플로우를 **시각화/디버깅/모니터링** 가능하며,  
  OpenAI의 평가/파인튜닝/디스틸레이션 툴과 통합 가능

In [4]:
# pip install openai-agents

In [1]:
import os
import openai
import sys
sys.path.append('./')

import logging
logging.getLogger("httpx").setLevel(logging.WARNING)

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv()) # read local .env file

Model = "gpt-4.1-mini"

### Hello World 예제

| 메서드                        | 호출 방식                   | 특징                                  |
| -------------------------- | ----------------------- | ----------------------------------- |
| `Runner.run(...)`          | `await Runner.run(...)` | 비동기적, 에이전트 루프 자동 실행, 도구 & 핸드오프 지원   |
| `Runner.run_sync(...)`     | `Runner.run_sync(...)`  | 동기 실행으로 첫 번째 메서드 래핑, 스크립트/테스트 환경 적합 |
| `Runner.run_streamed(...)` | 비동기 + 스트리밍              | 중간 응답을 이벤트로 실시간 전송 가능               |


In [2]:
from agents import Agent, Runner

agent = Agent(
    name="Assistant",
    instructions="당신은 도움되는 도우미입니다.",
    model=Model
)

# 비동기적으로 에이전트를 실행하여 사용자 요청에 대한 응답을 받음
# 요청: "재귀적 프로그래밍에 대한 짧은 시를 3줄 이내로 써주세요."
result = await Runner.run(starting_agent=agent, 
                          input="재귀적 프로그래밍에 대한 짧은 시를 3줄 이내로 써주세요.")

# 최종 응답 결과를 출력
print(result.final_output)

끝없는 호출,  
자기 안에 숨은 답,  
재귀의 마법.


### Simple Example

언어에 따라 적절한 에이전트에 작업을 위임(handoff)

In [3]:
from agents import Agent, Runner

# 한국어 에이전트 생성: 한국어만 사용 가능
korean_agent = Agent(
    name="한국어 agent",
    instructions="당신은 한국어만 할 수 있습니다.",
    model=Model
)

# 영어 에이전트 생성: 영어만 사용 가능
english_agent = Agent(
    name="영어 agent",
    instructions="당신은 영어만 할 수 있습니다.",
    model=Model
)

# 분류 역할의 핸드오프 에이전트 생성
# 입력된 문장의 언어를 판별하여 적절한 에이전트(한국어 or 영어)에게 전달
handoff_agent = Agent(
    name="분류 agent",
    instructions="요청에 사용된 언어에 따라 적절한 에이전트에게 넘겨주세요.",
    model=Model,
    handoffs=[korean_agent, english_agent],  # 연결할 하위 에이전트 목록
)

# Agent orchenstration 실행
result = await Runner.run(handoff_agent, input="폭싹 속았쑤까?")
print(result.final_output)  # 한국어 에이전트가 응답

result = await Runner.run(handoff_agent, input="Are you happy?")
print(result.final_output)  # 영어 에이전트가 응답

폭싹 속았쑤까? 라고 하셨는데, 상황을 좀 더 알려주시면 도와드릴 수 있을 것 같아요! 어떤 일 때문에 그렇게 느끼셨나요?
As an AI, I don't have feelings, so I don't experience happiness or sadness. But I'm here to help you! How can I assist you today?


----------------------------
**`Runner.run_streamed()`는 비동기로 실행되며 `RunResultStreaming`을 반환합니다.**  
이 함수는 LLM을 **스트리밍 모드**로 호출하며, **LLM으로부터 받은 응답을 실시간으로 이벤트 형태로 스트리밍하여 전달**합니다.

In [4]:
# 에이전트에게 스트리밍 방식으로 입력을 보내고 응답을 받음
response = Runner.run_streamed(
    starting_agent=agent,
    input="짧은 이야기를 해줘"  # 사용자 입력: 짧은 이야기 요청
)

# 에이전트로부터 스트리밍 응답 이벤트를 비동기적으로 반복
async for event in response.stream_events():
    # 응답 타입이 텍스트 델타(부분 응답)일 경우
    if event.type == "raw_response_event":
        data = event.data  # 이벤트 데이터 추출

        # 텍스트 출력 델타가 있는 경우 해당 부분을 출력
        if data.type == "response.output_text.delta":
            print(data.delta, end="", flush=True)  # 이어붙이면서 바로 출력

옛날 어느 작은 마을에 마음씨 착한 소녀 하나가 살고 있었어요. 어느 날, 숲 속에서 길을 잃은 작은 새를 발견했죠. 소녀는 새에게 물과 먹이를 주며 함께 집으로 데려갔어요. 며칠 후, 새는 힘을 되찾아 하늘 높이 날아갔고, 그 순간 하늘에서 반짝이는 작은 별이 떨어졌답니다. 소녀는 그 별을 간직하며, 언제나 작은 친절이 큰 기적을 만든다는 걸 잊지 않았어요.

------------------------
## 도구 (Tools)

**도구(Tools)** 는 에이전트가 **행동을 수행할 수 있도록 해주는 기능**입니다.  
예를 들어, **데이터를 가져오거나**, **코드를 실행하고**, **외부 API를 호출하거나**, 심지어 **컴퓨터를 사용하는 작업**까지도 가능합니다.

Agent SDK에서는 도구를 다음 **세 가지 유형**으로 구분합니다:  

### 1. **Hosted tools (호스팅 도구)**  
이 도구들은 **LLM 서버 내에서**, AI 모델과 함께 실행됩니다.  
OpenAI는 다음과 같은 호스팅 도구를 제공합니다:
- 검색 기반 정보 검색 (Retrieval)
- 웹 검색 (Web search)
- 컴퓨터 사용 (Computer use)

### 2. **Function calling (함수 호출 도구)**  
이 방식은 **Python 함수**를 도구로 등록하여 사용할 수 있게 해줍니다. 즉, 여러분이 직접 정의한 함수도 LLM이 도구처럼 호출할 수 있습니다.  

**Agents SDK**에서는 **모든 Python 함수를 도구(tool)** 로 사용할 수 있습니다. SDK는 이러한 도구를 자동으로 설정해줍니다:

1. **도구 이름**은 Python 함수의 이름에서 자동으로 생성됩니다.  

2. **도구 설명(description)** 은 함수의 **docstring**에서 가져옵니다.  

3. 함수 입력값의 **스키마(schema)** 는 함수의 **인자(arguments)** 로부터 자동 생성됩니다.

4. 각 입력값에 대한 **설명**도 함수의 docstring에서 추출됩니다.  

### 3. **Agents as tools (에이전트를 도구처럼 사용)**  
이 방식은 **하나의 에이전트를 도구로 사용**할 수 있게 해줍니다.  
즉, **핸드오프 없이도** 에이전트가 다른 에이전트를 호출할 수 있습니다.

-------------------------

`Runner.run()`을 호출하면, 최종 결과가 나올 때까지 루프가 실행됩니다.  이 루프의 동작 방식은 다음과 같습니다:

1. **LLM 호출:**  
   에이전트에 설정된 모델과 옵션, 그리고 대화 기록(message history)을 바탕으로 LLM을 호출.  

2. **LLM 응답 처리:**  
   모델이 응답을 반환하며, 여기에는 **툴 호출(tool calls)**이 포함될 수 있습니다.

3. **최종 출력이 있는 경우:**  
   응답에 **최종 출력(final output)**이 포함되어 있다면, 이를 반환하고 루프를 종료.

4. **핸드오프가 있는 경우:**  
   응답에 **다른 에이전트로의 핸드오프(handoff)**가 있으면, 현재 에이전트를 새로운 에이전트로 설정한 후 1단계부터 다시 시작.

5. **툴 호출 처리:**  
   툴 호출이 있을 경우, 해당 툴을 실행하고 결과 메시지를 기록한 다음 1단계로 되돌아갑니다.


**Final Output(최종 출력)** 은 루프에서 에이전트가 생성한 마지막 결과물입니다.

- 에이전트에 `output_type`이 설정되어 있다면:  
  → LLM이 해당 형식에 맞는 structured output(구조화된 출력)을 반환할 때 루프가 종료

- `output_type`이 설정되지 않은 경우:  
  → 툴 호출이나 핸드오프가 포함되지 않은 첫 번째 LLM 응답이 최종 출력으로 간주

In [5]:
# 함수 호출 example
from agents import function_tool
import requests

@function_tool
def multiply(x: float, y: float) -> float:
    """
    x 와 y 를 곱한다.
    """
    print("** multiply 함수 실행 **", x, y)
    return x * y

@function_tool
def get_weather(위도: float, 경도: float) -> str:
    print(위도, 경도)
    print(f"Weather 함수 실행 - 도시: {위도, 경도}")
    latitude=위도
    longitude=경도
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m")
    data = response.json()
    return data['current']['temperature_2m']

agent = Agent(
    name="Assistant",
    instructions=(
        "당신은 유용한 도우미 입니다. 가능하면 제공된 도구를 사용하세요."
        "당신은 기상 전문가 입니다. 도시의 경도와 위도를 주면 날씨를 알려줍니다."
        "사용자 질의에 답하기 위해 당신의 지식에만 너무 의존하지 말고 도구를 사용하세요."
    ),
    model="gpt-4o-mini",
    tools=[multiply, get_weather]
)

weather_agent = Agent(
    name="Weather Agent",
    instructions="당신은 기상 전문가 입니다. 도시의 경도와 위도를 주면 날씨를 알려줍니다.",
    model="gpt-4.1",
    tools=[get_weather]
)

result = await Runner.run(agent, input="7.814 곱하기 103.892의 값은?")
print(result.final_output)

** multiply 함수 실행 ** 7.814 103.892
7.814 곱하기 103.892의 값은 811.812088입니다.


In [6]:
result = await Runner.run(weather_agent, input="서울의 날씨는 어떻습니까?")
print(result.final_output)

37.5665 126.978
Weather 함수 실행 - 도시: (37.5665, 126.978)
현재 서울(위도 37.5665, 경도 126.978)의 기온은 약 24.2°C입니다. 추가적으로 알고 싶은 날씨 정보가 있으신가요? (예: 강수량, 미세먼지, 풍속 등)


In [7]:
result = await Runner.run(agent, input="서울의 날씨 곱하기 20은 얼마입니까?")
print(result.final_output)

37.5665 126.978
Weather 함수 실행 - 도시: (37.5665, 126.978)
** multiply 함수 실행 ** 24.2 20.0
서울의 날씨는 24.2도이며, 이를 20배하면 484.0입니다.


## Guardrails (가드레일)

**가드레일(Guardrails)** 은 에이전트와 **병렬로 실행되며**, 사용자 입력에 대해 **검사와 유효성 검증**을 수행할 수 있게 해줍니다.

예를 들어, 아주 똑똑하지만 **느리고 비용이 많이 드는 모델**을 사용하는 에이전트를 악의적인 사용자가 수학 숙제를 대신 풀어달라고 요청하는 경우,
빠르고 저렴한 모델을 활용한 가드레일을 실행하여, 사용자의 입력이 악의적인 목적(예: 숙제 대행)인지 먼저 검사할 수 있습니다.  
가드레일이 의심스러운 입력을 감지하면 즉시 오류를 발생시켜 고비용 모델의 실행을 막고 시간과 비용을 절약할 수 있습니다.

In [8]:
from agents import Agent, InputGuardrail, GuardrailFunctionOutput, Runner
from pydantic import BaseModel

Model = "o3-mini"

# guardrail 에이전트의 출력 형식을 정의
# is_homework: 숙제인지 여부
# reasoning: 숙제로 판단한 이유 설명
class HomeworkOutput(BaseModel):
    is_homework: bool          # 이 입력이 숙제 질문인지 여부
    reasoning: str             # 판단 이유 설명

In [9]:
# 숙제 여부를 판단하는 guardrail 전용 에이전트 정의
guardrail_agent = Agent(
    name="Guardrail check",  # 에이전트 이름
    instructions="사용자가 숙제 질문을 하고 있는지 확인하세요.",  # LLM에게 줄 지시문
    model=Model,
    output_type=HomeworkOutput,  # 결과는 HomeworkOutput 형식으로 반환
)

# 수학 질문 전용 튜터 에이전트 정의
math_tutor_agent = Agent(
    name="Math Tutor",  # 에이전트 이름
    handoff_description="수학 질문을 위한 전문 에이전트",  # 다른 에이전트가 handoff할 때 참고하는 설명
    model=Model,
    instructions="당신은 수학 문제를 도와주는 튜터입니다. 각 단계의 이유를 설명하고 예시를 포함하세요.",  # LLM 지시문
)

# 역사 질문 전용 튜터 에이전트 정의
history_tutor_agent = Agent(
    name="History Tutor",  # 에이전트 이름
    handoff_description="역사 질문을 위한 전문 에이전트",  # 다른 에이전트가 handoff할 때 참고하는 설명
    model=Model,
    instructions="당신은 역사 질문을 도와주는 튜터입니다. 중요한 사건들과 그 맥락을 명확히 설명하세요.",  # LLM 지시문
)

# 입력이 '숙제 질문'인지 판단하는 guardrail 함수 정의
# triage_agent에서 InputGuardrail로 사용됨
async def homework_guardrail(ctx, agent, input_data):
    # guardrail_agent를 실행하여 숙제 여부 판단
    result = await Runner.run(guardrail_agent, input_data, context=ctx.context)

    # 결과를 HomeworkOutput 모델 형태로 변환
    final_output = result.final_output_as(HomeworkOutput)

    # GuardrailFunctionOutput을 반환
    # is_homework가 False이면 tripwire_triggered를 True로 설정하여 실행을 중단시킴
    return GuardrailFunctionOutput(
        output_info=final_output,               # guardrail 판단 결과
        tripwire_triggered=not final_output.is_homework,  # 숙제가 아니면 트리거 작동
    )

In [10]:
# 사용자 질문이 숙제인지 확인하고,
# 숙제라면 수학 튜터 또는 역사 튜터 에이전트로 전달(handoff)하는 판단 에이전트 정의
handoff_agent = Agent(
    name="Triage Agent",  # 에이전트 이름
    instructions="사용자의 숙제 질문을 기반으로 어떤 에이전트를 사용할지 결정하세요.",  
    handoffs=[history_tutor_agent, math_tutor_agent],  # 질문을 위임할 수 있는 대상 에이전트 목록
    input_guardrails=[
        InputGuardrail(guardrail_function=homework_guardrail),  # 입력값에 대해 사전 검사를 수행하는 가드레일
    ],
)

# triage_agent를 테스트하는 비동기 함수 정의
async def question(prompt):
    try:
        # 첫 번째 테스트: 역사 관련 숙제 질문
        result = await Runner.run(handoff_agent, prompt)
        print("Output:", result.final_output)  # 출력 결과 출력
    except Exception as e:
        print("Guardrail에 의해 입력이 거부되었습니다:", e)  # guardrail이 작동하면 예외 메시지 출력

# main 함수 실행 (Notebook 또는 async 환경에서 사용 가능)
prompt = "고구려의 첫번째 왕은 누구인가요?"
await question(prompt)

print("-----------------------------------------------------------------------------------------------------------------------------")

prompt = "사과가 상자에 10개가 담겨 있습니다. 5개를 먹고 3개를 추가하면 상자에는 몇개가 남습니까. 단계별로 생각하세요."
await question(prompt)

print("-----------------------------------------------------------------------------------------------------------------------------")

prompt = "사과와 감 중에 어느 것이 더 달아?"
await question(prompt)

Output: 고구려의 첫 번째 왕은 주몽(朱蒙)입니다.

주몽은 기원전 37년경 고구려를 건국한 인물로 전해지며, 그의 전설과 일대기는 삼국유사와 삼국사기 등 주요 역사서에 기록되어 있습니다. 주몽은 여러 신화와 전설로 둘러싸인 인물로, 용맹과 지략을 겸비한 지도자로 묘사되며, 이후 그가 세운 고구려는 강력한 군사력과 영토 확장을 통해 한반도 북부와 만주 일대를 지배하게 됩니다.

주몽의 업적은 고구려가 장기간 한반도와 동북아시아 지역에서 강국으로 성장할 수 있는 초석을 마련했다는 점에서 중요한 의미를 지니고 있습니다. 그의 건국 정신과 지도력은 후대에 큰 영향을 미쳤으며, 고구려 문화와 국가 정체성의 기초로 평가받고 있습니다.
-----------------------------------------------------------------------------------------------------------------------------
Output: 문제: 사과가 상자에 10개가 담겨 있습니다. 5개를 먹고 3개를 추가하면 상자에는 몇 개가 남습니까?

단계별 풀이:

1. 처음 사과의 개수는 10개입니다.
   - 예시: 상자 안에 사과 10개가 들어있다고 생각해봅시다.

2. 5개를 먹습니다. 먹으면 사과의 수가 줄어듭니다.
   - 계산: 10 - 5 = 5
   - 예시: 만약 친구와 10개의 사과를 가지고 있었고, 5개를 먹었다면 남는 사과는 5개입니다.

3. 3개를 추가합니다. 추가하면 사과의 수가 늘어납니다.
   - 계산: 5 + 3 = 8
   - 예시: 남은 5개의 사과에 3개의 사과를 더 넣으면 상자에는 8개의 사과가 있게 됩니다.

따라서 최종적으로 상자에 남은 사과의 개수는 8개입니다.
-----------------------------------------------------------------------------------------------------------------------------
Guardr

### 구조화된 출력

In [11]:
from pydantic import BaseModel
from agents import Agent, Runner

# 캘린더 이벤트 데이터 구조 정의
class CalendarEvent(BaseModel):
    name: str              # 이벤트 이름
    date: str              # 이벤트 날짜
    participants: list[str]  # 참가자 목록

# 에이전트 정의: 텍스트에서 캘린더 이벤트 정보를 추출
agent = Agent(
    name="캘린더 추출기",  # 에이전트 이름
    instructions="텍스트에서 캘린더 이벤트를 추출하세요. "
                 "이벤트 이름, 날짜, 참가자 정보를 구조화된 데이터로 반환하세요.",
    output_type=CalendarEvent,  # 반환 데이터 형식
    model="gpt-4.1-mini",             # 사용할 LLM 모델
)

# 실행 함수 정의
async def main():
    input_text = (
        "2025년 4월 10일에 '분기 전략 회의'라는 이름의 팀 미팅이 예정되어 있습니다. "
        "참가자는 오길동, 한철수, 김미미입니다."
    )

    result = await Runner.run(agent, input_text)  # 에이전트 실행
    print("추출된 캘린더 이벤트:")
    print(result.final_output)  # 결과 출력

# 비동기 함수 실행
await main()

추출된 캘린더 이벤트:
name='분기 전략 회의' date='2025-04-10' participants=['오길동', '한철수', '김미미']


### 실습 문제

1. **분류 에이전트**:  
   사용자 입력이 \*\*‘수학 문제’\*\*인지 \*\*‘기타 질문’\*\*인지 분류하세요.  
   분류 결과에 따라 적절한 에이전트로 handoff 하도록 하세요.

3. **수학 에이전트**:  
   수학 문제일 경우, Python 함수 `calculate_area(length: float, width: float)`를 도구로 등록하여 `직사각형의 넓이`를 계산해주는 역할을 수행하세요.

4. **일반 에이전트**:  
   기타 질문에 대해서는 "질문을 이해했지만 수학 관련 질문만 도와드릴 수 있어요." 라고 응답하세요.

5. **가드레일**:  
   입력 내용에 \*\*금지어(`해킹`, `폭탄`)\*\*가 포함되어 있으면 guardrail을 작동시켜 에이전트 실행을 **중단**시키세요.  
   (힌트: InputGuardrail 사용)


### 테스트 입력 예시

* `"가로 5, 세로 7인 직사각형의 넓이를 구해주세요."`  
  👉 수학 에이전트 → 함수 실행 → 넓이 출력  

* `"어제 뉴스에 나온 해킹 사고에 대해 말해줘"`  
  👉 가드레일 트리거 → 실행 중단  

* `"오늘 날씨 어때?"`  
  👉 일반 에이전트 응답